In [12]:
import random as rn

In [1]:
def btoi(x):
    return int.from_bytes(x, 'little')

In [161]:
## First, some preliminaries that will be needed.

import hashlib

def sha512(s):
    return hashlib.sha512(s).digest()

# Base field Z_p
p = 2**255 - 19

def modp_inv(x):
    return pow(x, p-2, p)

# Curve constant
d = -121665 * modp_inv(121666) % p

# Group order
q = 2**252 + 27742317777372353535851937790883648493

def sha512_modq(s):
    hsh = sha512(s)
    print("HSH: ", hsh.hex())
    return int.from_bytes(hsh, "little") % q

## Then follows functions to perform point operations.

# Points are represented as tuples (X, Y, Z, T) of extended
# coordinates, with x = X/Z, y = Y/Z, x*y = T/Z

def point_inv(X):
    return (0 - X[0]) % p, X[1], X[2], ((0 - X[3]) % p)

def point_add(P, Q):
    A, B = (P[1]-P[0]) * (Q[1]-Q[0]) % p, (P[1]+P[0]) * (Q[1]+Q[0]) % p
    C, D = 2 * P[3] * Q[3] * d % p, 2 * P[2] * Q[2] % p
    E, F, G, H = B-A % p, D-C % p, D+C % p, B+A % p
    return (E*F % p, G*H % p, F*G % p, E*H % p)

def point_sub(P, Q):
    return point_add(P, point_inv(Q))

def point_dub(X):
    A = X[0]**2 % p
    B = X[1]**2 % p
    C = 2*(X[2]**2) % p
    H = A+B % p
    E = H-(X[0]+X[1])**2 % p
    G = A-B % p
    F = C+G % p
    X3 = E*F % p
    Y3 = G*H % p
    T3 = E*H % p
    Z3 = F*G % p
    return (X3, Y3, Z3, T3)

def swap(P, Q):
    return Q, P

# Computes Q = s * Q
def point_mul(s, P):
    Q = (0, 1, 1, 0)  # Neutral element
    for i in range(256, -1, -1):
        if s & (1 << i):
            P, Q = swap(P, Q)
        P = point_add(Q, P)
        Q = point_dub(Q)
        if s & (1 << i):
            P, Q = swap(P, Q)
    return Q

def point_equal(P, Q):
    # x1 / z1 == x2 / z2  <==>  x1 * z2 == x2 * z1
    if (P[0] * Q[2] - Q[0] * P[2]) % p != 0:
        return False
    if (P[1] * Q[2] - Q[1] * P[2]) % p != 0:
        return False
    return True

## Now follows functions for point compression.

# Square root of -1
modp_sqrt_m1 = pow(2, (p-1) // 4, p)

# Compute corresponding x-coordinate, with low bit corresponding to
# sign, or return None on failure
def recover_x(y, sign):
    if y >= p:
        return None
    x2 = (y*y-1) * modp_inv(d*y*y+1)
    if x2 == 0:
        if sign:
            return None
        else:
            return 0

    # Compute square root of x2
    x = pow(x2, (p+3) // 8, p)
    if (x*x - x2) % p != 0:
        x = x * modp_sqrt_m1 % p
    if (x*x - x2) % p != 0:
        return None

    if (x & 1) != sign:
        x = p - x
    return x

# Base point
g_y = 4 * modp_inv(5) % p
g_x = recover_x(g_y, 0)
G = (g_x, g_y, 1, g_x * g_y % p)

def point_compress(P):
    zinv = modp_inv(P[2])
    x = P[0] * zinv % p
    y = P[1] * zinv % p
    return int.to_bytes(y | ((x & 1) << 255), 32, "little")

def point_decompress(s):
    if len(s) != 32:
        raise Exception("Invalid input length for decompression")
    y = int.from_bytes(s, "little")
    sign = y >> 255
    y &= (1 << 255) - 1

    x = recover_x(y, sign)
    if x is None:
        return None
    else:
        return (x, y, 1, x*y % p)

def point_print(X):
    zinv = modp_inv(X[2])
    x = X[0] * zinv % p
    y = X[1] * zinv % p
    print(hex(x), hex(y))

def point_print_full(X):
    print(hex(X[0]), hex(X[1]), hex(X[2]), hex(X[3]))

## These are functions for manipulating the private key.

def secret_expand(secret):
    if len(secret) != 32:
        raise Exception("Bad size of private key")
    h = sha512(secret)
    a = int.from_bytes(h[:32], "little")
    a &= (1 << 254) - 8
    a |= (1 << 254)
    return (a, h[32:])

def secret_to_public(secret):
    (a, dummy) = secret_expand(secret)
    return point_compress(point_mul(a, G))

## The signature function works as below.

def sign(secret, msg):
    a, prefix = secret_expand(secret)
    A = point_compress(point_mul(a, G))
    r = sha512_modq(prefix + msg)
    R = point_mul(r, G)
    Rs = point_compress(R)
    h = sha512_modq(Rs + A + msg)
    s = (r + h * a) % q
    return Rs + int.to_bytes(s, 32, "little")

## And finally the verification function.

def verify(public, msg, signature):
    if len(public) != 32:
        raise Exception("Bad public key length")
    if len(signature) != 64:
        Exception("Bad signature length")
    A = point_decompress(public)
    point_print_full(A)
    if not A:
        return False
    Rs = signature[:32]
    R = point_decompress(Rs)
    if not R:
        return False
    s = int.from_bytes(signature[32:], "little")
    print("S:", hex(s))
    if s >= q: return False
    toh = Rs + public + msg
    print("TOH:", toh.hex())
    h = sha512_modq(Rs + public + msg)
    print("h:", hex(h))
    sB = point_mul(s, G)
    hA = point_mul(h, A)
    point_print_full(sB)
    point_print_full(hA)
    xxx = point_sub(sB, hA)
    point_print(xxx)
    sBmhA = point_compress(xxx)
    print("")
    print("R   :", Rs.hex())
    print("RES : ", sBmhA.hex())
    print(Rs == sBmhA)
    return point_equal(sB, point_add(R, hA))

In [135]:
hex(d)

'0x52036cee2b6ffe738cc740797779e89800700a4d4141d8ab75eb4dca135978a3'

In [11]:
import sys
import binascii

def munge_string(s, pos, change):
    return (s[:pos] +
            int.to_bytes(s[pos] ^ change, 1, "little") +
            s[pos+1:])

# Read a file in the format of
# http://ed25519.cr.yp.to/python/sign.input
line = eddsa_in
fields = line.split(":")
secret = (binascii.unhexlify(fields[0]))[:32]
public = binascii.unhexlify(fields[1])
msg = binascii.unhexlify(fields[2])
signature = binascii.unhexlify(fields[3])[:64]

pubkey = secret_to_public(secret)

print("public:", hex(btoi(public)))
print("msg:", hex(btoi(msg)))
print("signature:", hex(btoi(signature)))
print()

assert public == pubkey
assert signature == sign(secret, msg)
assert verify(public, msg, signature)
if len(msg) == 0:
    bad_msg = b"x"
else:
    bad_msg = munge_string(msg, len(msg) // 3, 4)
#assert not verify(public,bad_msg,signature)
#assert not verify(public, msg, munge_string(signature,20,8))
#assert not verify(public,msg,munge_string(signature,40,16))

public: 0xff4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b231
msg: 0x91f61b8cedad86a7d733
signature: 0x8ced65d6a69a67f49870a9f5ada999e258894367c6237391983f0404c1a15d909949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428

A: 0x4cd530baaf96cfce24c4baf37493d3cc26d6d2c886c8d8938da2c0c48aaa27ab 0x7f4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b231 0x1
Rs: 0x9949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428
S: 0x8ced65d6a69a67f49870a9f5ada999e258894367c6237391983f0404c1a15d9
TOH: 0x91f61b8cedad86a7d733ff4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b23109949c55e6b9796edf0c2bdd37f58b9e914eab3f9f3ff001dee95a418c59e428
h: 0xc41c35527fc3ecea13195c06574ced8b3e32a3b5ea082e240e3aa36aadcd50a


In [43]:
msg = rn.randint(0, 2**512-1).to_bytes(64, 'little')

In [53]:
msg = "8f13b3a29344b73d22e681e9faeb3fedb88c94f7504f8f2ac2f17a09fc33a1f6b83275219d83def87aa1f74537c8819db6759c80fff5b4c42aa09b663c5304d9"
msg = binascii.unhexlify(msg)

In [162]:
signature = sign(secret, msg)

print("MSG  :", msg.hex())
print("SIGN :", signature.hex())

public = secret_to_public(secret)
print("PUB  :", public.hex())
print(" -- ")
verify(public, msg, signature)


HSH:  a612c0a22b876d35171ff7a3a6f92c211e5fa808b8e1b67567239053d428d40720821bb245fe557caee399665421299619ba8370228c38e8349e9a7787c6a849
HSH:  702cab421b73a2fc689bd146f1cc09dbd04e6906de658a452325f2284e5eb4cd201deaf8e6523db22e0e88a433ab00554ccde61d7e610d257d89087b8a98eea2
MSG  : 8f13b3a29344b73d22e681e9faeb3fedb88c94f7504f8f2ac2f17a09fc33a1f6b83275219d83def87aa1f74537c8819db6759c80fff5b4c42aa09b663c5304d9
SIGN : 2002052f204fc13574a176de46fd6f092b2f5c33a3c704a47595cf97eef049abb8b7fdf9e21690f0def687edb4925a9e3de6e03bfe78530c166425e7898f7809
PUB  : 31b2524b8348f7ab1dfafa675cc538e9a84e3fe5819e27c12ad8bbc1a36e4dff
 -- 
0x4cd530baaf96cfce24c4baf37493d3cc26d6d2c886c8d8938da2c0c48aaa27ab 0x7f4d6ea3c1bbd82ac1279e81e53f4ea8e938c55c67fafa1dabf748834b52b231 0x1 0x7f06d1d06a863df81772760e593646724c7288d419e5ecb4fbf1bd2af579a996
S: 0x9788f89e72564160c5378fe3be0e63d9e5a92b4ed87f6def09016e2f9fdb7b8
TOH: 2002052f204fc13574a176de46fd6f092b2f5c33a3c704a47595cf97eef049ab31b2524b8348f7ab1dfafa675cc538e9a84e3f

True

In [158]:
Q1 = (
    0x7ac4a5340c0cd2a7f32d7b47ce4593d11b346696da854f1fe299cfcbf8be8edc,
    0x24584354f7e1b9c4a7991dd930d26b13b4e1d107151e056f314a44544a9e54f7,
    0x74216e7931100caaaa25fdda763c404974d9987d25631fec0d26b635428b0b6c,
    0x578adb28e88d1416e0e615ad55b1a2d97f14bc1707dcd857ee511f159388d995
)

Q2 = (
    0x71396c2da6770bc5e58d761c8a180416efabdcd60ca7070dee60e4a21ead0ad8,
    0x5b23871c0192c6773c2eca8cee8af0b3f2a9b6f743675fd050b291d59333ab65,
    0x4d822f287799d0b804caf7ef466ba82ebf75019a5dbabcd595f5f1d1e8961af2,
    0x43fbad63c42b173221031290f19f3b4703516530c408e637f551c47ff8fd5553
)

point_equal(Q1, Q2)

True

In [150]:
point_print_full(point_mul(2, G))

0x1ec11736254444f76a4070c6f32cd4999437afef4b972189ff2fb86ab3644256 0x11b13d7170b9e22a8586a34b2a7e3b703db728a843fac0fb0a7a6b0f159ee87f 0x7396af2b24cbdaa1cf01606607a7c2d514c3511f0e5df106353c31e0e179794b 0x27a1e4cd8ac902b65240ac787d51bbc5b82e9d950e0d78ed3e2dd58456506734


In [149]:
point_equal(point_mul(2, G), point_add(G, G))

True

In [151]:
point_print_full(point_dub((0,1,1,0)))

0x0 0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffec 0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffec 0x0


In [105]:
point_print(point_add(G, G))
point_print(point_dub((G)))

0x36ab384c9f5a046c3d043b7d1833e7ac080d8e4515d7a45f83c5a14e2843ce0e 0x2260cdf3092329c21da25ee8c9a21f5697390f51643851560e5f46ae6af8a3c9
0x36ab384c9f5a046c3d043b7d1833e7ac080d8e4515d7a45f83c5a14e2843ce0e 0x2260cdf3092329c21da25ee8c9a21f5697390f51643851560e5f46ae6af8a3c9


In [106]:
point_equal(point_add(G, G), point_dub((G)))

True

In [49]:
kio = point_dub((0, 1, 1, 0))
print(hex(kio[0]), hex(kio[1]), hex(kio[2]), hex(kio[3]))

0x0 0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffec 0x0 0x1


In [140]:
point_print_full(G)

0x216936d3cd6e53fec0a4e231fdd6dc5c692cc7609525a7b2c9562d608f25d51a 0x6666666666666666666666666666666666666666666666666666666666666658 0x1 0x67875f0fd78b766566ea4e8e64abe37d20f09f80775152f56dde8ab3a5b7dda3


In [143]:
point_print_full(point_add((0,1,1,0), G))

0x44fd2f9298f81267a5c18434688f8a09fd399f05d140beb39d103905d740913e
0x7cf9d3a33d4ba65270b4898643d42c2cf932dc6fb8c0e192fbc93c6f58c3b85
0x0
0x2
-0x3d2d9258652358027eb63b9c045247472da6713ed5b4b09a6d53a53ee1b455b9
0x2
0x2
0x4cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc3
0x5a4db4f35b94ffb029388c7f75b7171a4b31d8254969ecb2558b5823c97547b 0x1999999999999999999999999999999999999999999999999999999999999999 0x4 0x1e1d7c3f5e2dd9959ba93a3992af8df483c27e01dd454bd5b77a2ace96df76c5


In [153]:
T = (
    0x5dfa71625b7aba864c18300a5e4b5c6e98ee6d891ec11736254444f76a4070b9,
    0x662bbe321c924b2cd95e2d7fde976e9b5825ddb291b13d7170b9e22a8586a33c,
    0x00339299d16374a93d0278de7aaa015005eefc74f396af2b24cbdaa1cf016066,
    0x3fedf0bb190eb34e071b4d499c956aabe752f53e27a1e4cd8ac902b65240ac6f
)
point_equal(T, point_mul(2, G))

True